# Exploratory Analysis – Gene Prioritization (PhenoGenius 2022)

## Objectives
This notebook presents a descriptive analysis of gene prioritization results
for **a given method**, focusing on:

- the cohort of patients analyzed
- the genes and variants involved
- the distribution of ACMG and ClinVar classifications
- the single-gene/multi-variant structure
- an initial overview of prioritization performance

This notebook compares **only one run** (tool × HPO × extraction).

## Imports and parameters

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ast


pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 120)

# === Parameters ===
INPUT_PATH = "../data/analysis_table.csv"   # ou .parquet
INPUT_PATH_EHOP = "../data/ehop_one_gene_export_with_keys.tsv"   # ou .parquet
OUTPUT_DIR = "../figures"
os.makedirs(OUTPUT_DIR, exist_ok=True)

RUN_ID = "phenogenius_hpo2022_sm"


## Loading parameters

In [13]:
run_df = pd.read_csv(INPUT_PATH_EHOP, sep="\t", index_col=0)
run_df = run_df.dropna(subset=["transcript"])
run_df.shape

(415, 20)

In [ ]:
# Loading
if INPUT_PATH.endswith(".parquet"):
    df = pd.read_parquet(INPUT_PATH)
else:
    df = pd.read_csv(INPUT_PATH)

print("Total rows in table :", len(df))
df.head()


Total lignes dans la table : 5


,ID_PAT_ETUDE,IPP,IPP_clef,key_for_chaining,transcript_from_ehop,transcript,transcript_no_version,chr,pos,ref,alt,var_id,transcript_nm,gene_symbol,gene_hgnc_id,acmg_classification,acmg_criteria,clinvar_classification,clinvar_review_status,run_id,prio_tool,hpo_version,extraction_method,report_path,report_found,report_read_error,report_read_error_msg,gene_found_in_report,gene_not_found_flag,gene_duplicated_in_report,rank,score,hpo_implicated,hpo_description_implicated,phenotype_specificity,top1,top3,top5,top10
0,report159,IPP000159,A1B2,report159,NM_000138.5,NM_000138.5,NM_000138,1,1234567,G,A,var159,NM_000138,FLNA,HGNC:3754,Pathogenic,"PVS1,PM2",Pathogenic,reviewed_by_expert_panel,phenogenius_hpo2022_sm,phenogenius,2022,string_matching,ext/phenogenius_results_analysis/data/fake_phenogenius_output/report159.tsv,True,False,NaN,True,False,False,6,8.12,"[{'HP:0001658': 0.2}, {'HP:0001155': 0.5}, {'HP:0001387': 0.7}, {'HP:0001760': 0.4}, {'HP:0002814': 0.5}, {'HP:00016...","[{'Myocardial infarction': 0.2}, {'Abnormality of the hand': 0.5}, {'Joint stiffness': 0.7}, {'Abnormal foot morphol...","A - the reported phenotype is highly specific and relatively unique to the gene (top 40, 50 perc of diagnosis in Phe...",False,False,False,True
1,report27,IPP000027,C3D4,report27,NM_001127500.3,NM_001127500.3,NM_001127500,X,7654321,C,T,var27,NM_001127500,MECP2,HGNC:6990,Likely_pathogenic,"PS2,PM2",Likely_pathogenic,criteria_provided_single_submitter,phenogenius_hpo2022_sm,phenogenius,2022,string_matching,ext/phenogenius_results_analysis/data/fake_phenogenius_output/report27.tsv,True,False,NaN,True,False,False,57,3.71,"[{'HP:0002624': 0.2}, {'HP:0001270': 0.6}, {'HP:0030693': 0.2}, {'HP:0012286': 0.2}, {'HP:0001317': 0.2}, {'HP:00023...","[{'Abnormal venous morphology': 0.2}, {'Motor delay': 0.6}, {'Supratentorial neoplasm': 0.2}, {'Abnormal hypothalamu...","B - the reported phenotype is consistent with the gene, is highly specific, but not necessarily unique to the gene (...",False,False,False,False
2,report32,IPP000032,E5F6,report32,NM_001042351.2,NM_001042351.2,NM_001042351,15,9876543,T,G,var32,NM_001042351,UBE3A,HGNC:12496,Pathogenic,"PVS1,PS3",Pathogenic,reviewed_by_expert_panel,phenogenius_hpo2022_sm,phenogenius,2022,string_matching,ext/phenogenius_results_analysis/data/fake_phenogenius_output/report32.tsv,True,False,NaN,True,False,False,361,2.39,"[{'HP:0002019': 0.5}, {'HP:0002360': 0.2}, {'HP:0001250': 0.3}, {'HP:0001382': 0.2}, {'HP:0002650': 0.2}, {'HP:00012...","[{'Constipation': 0.5}, {'Sleep disturbance': 0.2}, {'Seizure': 0.3}, {'Joint hypermobility': 0.2}, {'Scoliosis': 0....","C - the phenotype is reported with limited association with the gene, not highly specific and/or with high genetic h...",False,False,False,False
3,report34,IPP000034,G7H8,report34,NM_020975.6,NM_020975.6,NM_020975,7,2468101,A,C,var34,NM_020975,KMT2D,HGNC:7133,Likely_pathogenic,"PS4,PM1",Likely_pathogenic,criteria_provided_multiple_submitters_no_conflicts,phenogenius_hpo2022_sm,phenogenius,2022,string_matching,ext/phenogenius_results_analysis/data/fake_phenogenius_output/report34.tsv,True,False,NaN,True,False,False,503,1.83,"[{'HP:0002650': 0.2}, {'HP:0001382': 0.2}, {'HP:0002360': 0.2}, {'HP:0001999': 0.4}, {'HP:0001250': 0.2}]","[{'Scoliosis': 0.2}, {'Joint hypermobility': 0.2}, {'Sleep disturbance': 0.2}, {'Abnormal facial shape': 0.4}, {'Sei...","C - the phenotype is reported with limited association with the gene, not highly specific and/or with high genetic h...",False,False,False,False
4,report54,IPP000054,I9J0,report54,NM_000088.3,NM_000088.3,NM_000088,17,1357911,G,T,var54,NM_000088,COL1A1,HGNC:2197,Pathogenic,"PVS1,PM5",Pathogenic,reviewed_by_expert_panel,phenogenius_hpo2022_sm,phenogenius,2022,string_matching,ext/phenogenius_results_analysis/data/fake_phenogenius_output/report54.tsv,True,False,NaN,True,False,False,29,5.65,"[{'HP:0001382': 0.9}, {'HP:0002617': 0.2}, {'HP:0002650': 0.6}, {'HP:0001369': 0.2}, {'HP:0001167': 0.2}, {'HP:00049...","[{'J

## Run selection

In [24]:
run_df = df[df["run_id"] == RUN_ID].copy()

print("Analysed run :", RUN_ID)
print("Number of rows :", len(run_df))
print("Number of unique patients :", run_df["ID_PAT_ETUDE"].nunique())


KeyError: 'run_id'

## General description of the cohort

In [4]:
n_patients = run_df["ID_PAT_ETUDE"].nunique()
n_genes = run_df["gene_symbol"].nunique()
n_variants = run_df["var_id"].nunique()

print(f"Analysed patients : {n_patients}")
print(f"Distincts genes : {n_genes}")
print(f"Distinct variants : {n_variants}")


Analysed patients : 379
Distincts genes : 270
Distinct variants : 396


## Mono-gene / multi-variants

Question: **How many patients have one gene for multiple variants on this gene?**

In [5]:
variants_per_patient = (
    run_df
    .groupby(["ID_PAT_ETUDE", "gene_symbol"])["var_id"]
    .nunique()
    .reset_index(name="n_variants")
    .sort_values(by="n_variants", ascending=False)
)

variants_per_patient.head()


,ID_PAT_ETUDE,gene_symbol,n_variants
309,DA68BC4AEC6CAB4F2F740B7D9157F4CEF1C56F19,BTD,2
127,6358D65E790749CED52D9069294C35DB5A8F5B0D,PRMT7,2
364,FA19440D5F1489A0FA8CE4AF59BB108426DF7427,PRG4,2
84,3B0E9FCC93EE9683E8B4DA9EEFF99F9C39D4ED5F,CPT2,2
305,D92331DCD20D66279343C7E0651C76828B2C4932,IFT172,2


In [6]:
multi_variant_patients = variants_per_patient[variants_per_patient["n_variants"] > 1]

print("Patients mono-gène avec ≥2 variants :", multi_variant_patients["ID_PAT_ETUDE"].nunique())
multi_variant_patients


Patients mono-gène avec ≥2 variants : 34


,ID_PAT_ETUDE,gene_symbol,n_variants
309,DA68BC4AEC6CAB4F2F740B7D9157F4CEF1C56F19,BTD,2
127,6358D65E790749CED52D9069294C35DB5A8F5B0D,PRMT7,2
364,FA19440D5F1489A0FA8CE4AF59BB108426DF7427,PRG4,2
84,3B0E9FCC93EE9683E8B4DA9EEFF99F9C39D4ED5F,CPT2,2
305,D92331DCD20D66279343C7E0651C76828B2C4932,IFT172,2
174,8212843D29682E3DF31F1C793FADDED2A94EFD3A,FOXRED1,2
232,A4C796B5D170F183C499CFAC46C98012F448A3AA,PCNT,2
94,43A917D224633514F1517B84C24F576949F316C9,POLR3A,2
26,117ADB48CB801984163164517A3BBA5AA6271E53,CFTR,2
303,D338BB0BDE4ADF8E86B26534CD6D9D1C880912A8,OSGEP,2


## ACMG Distribution (patients and genes)

#### Per patient (variant-level)

In [ ]:
acmg_patients_df = (
    run_df.drop_duplicates(subset=["ID_PAT_ETUDE"])
          .groupby("acmg_classification")["ID_PAT_ETUDE"]
          .nunique()
          .reset_index(name="n_patients")
          .sort_values("n_patients", ascending=False)
)

fig, ax = plt.subplots()

ax.bar(acmg_patients_df["acmg_classification"], acmg_patients_df["n_patients"])
ax.set_title("Distribution ACMG (patients)")
ax.set_xlabel("ACMG")
ax.set_ylabel("Nombre de patients")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "acmg_patients.png"), dpi=300)
plt.close(fig)



#### Per gene

In [ ]:
# acmg_genes = (
#     run_df
#     .drop_duplicates(subset=["gene_symbol"])
#     .groupby("acmg_classification")["gene_symbol"]
#     .nunique()
#     .sort_values(ascending=False)
# )

# acmg_genes

acmg_classification
Pathogenic                125
Uncertain_significance     75
Likely_pathogenic          55
Likely_benign              10
Benign                      5
Name: gene_symbol, dtype: int64

## Distribution ClinVar (patients)

#### Clinvar classification

In [ ]:
clinvar_patients_df = (
    run_df.drop_duplicates(subset=["ID_PAT_ETUDE"])
          .groupby("clinvar_classification")["ID_PAT_ETUDE"]
          .nunique()
          .reset_index(name="n_patients")
          .sort_values("n_patients", ascending=False)
)

fig, ax = plt.subplots()

ax.bar(clinvar_patients_df["clinvar_classification"], clinvar_patients_df["n_patients"])
ax.set_title("Distribution ClinVar classification (patients)")
ax.set_xlabel("ClinVar")
ax.set_ylabel("Nombre de patients")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "clinvar_patients.png"), dpi=300)
plt.close(fig)



#### Crossing ACMG x Clinvar

In [9]:
acmg_clinvar = (
    run_df
    .drop_duplicates(subset=["ID_PAT_ETUDE"])
    .pivot_table(
        index="acmg_classification",
        columns="clinvar_classification",
        values="ID_PAT_ETUDE",
        aggfunc="nunique",
        fill_value=0
    )
)

acmg_clinvar

clinvar_classification,"risk factor,Conflicting classifications of pathogenicity",Conflicting classifications of pathogenicity,Likely benign,Likely pathogenic,Pathogenic,Pathogenic/Likely pathogenic,Uncertain significance
acmg_classification,,,,,,,
Benign,1,2,2,0,0,0,0
Likely_benign,0,2,1,0,0,0,9
Likely_pathogenic,0,5,0,8,6,4,8
Pathogenic,0,2,0,6,64,35,0
Uncertain_significance,0,7,0,2,0,0,23


## Priorisation results quality

In [ ]:
flags = pd.DataFrame([{
    "report_found": int(run_df["report_found"].sum()),
    "report_read_error": int(run_df["report_read_error"].sum()),
    "gene_not_found": int(run_df["gene_not_found_flag"].sum()),
    "gene_duplicated": int(run_df["gene_duplicated_in_report"].sum()),
    "total_rows": len(run_df),
}])

flags_long = flags.melt(var_name="metric", value_name="count")
flags_long = flags_long[flags_long["metric"] != "total_rows"]

## PLOT
fig, ax = plt.subplots()

ax.bar(flags_long["metric"], flags_long["count"])
ax.set_title("Qualité des résultats (comptes)")
ax.set_ylabel("Count")
ax.set_xlabel("")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "quality_flags.png"), dpi=300)
plt.close(fig)

KeyError: 'report_found'

## Distribution of top-N ranks

In [15]:
rank_stats = run_df.loc[run_df["gene_found_in_report"], "rank"].describe()
rank_stats

count      5.00000
mean     191.20000
std      226.20168
min        6.00000
25%       29.00000
50%       57.00000
75%      361.00000
max      503.00000
Name: rank, dtype: float64

In [ ]:
topN = {
    "Top1": run_df["top1"].mean(),
    "Top3": run_df["top3"].mean(),
    "Top5": run_df["top5"].mean(),
    "Top10": run_df["top10"].mean(),
}

pd.Series(topN).apply(lambda x: f"{x:.1%}")

Top1      0.0%
Top3      0.0%
Top5      0.0%
Top10    20.0%
dtype: object

In [ ]:
# Mean rank even if monogenic here.
rank_eval = (
    run_df[run_df["report_found"] & (~run_df["report_read_error"])]
    .groupby("ID_PAT_ETUDE")["rank"]
    .mean()
    .reset_index(name="avg_rank")
)

## CDF plot sur avg_rank (≤ k)

In [ ]:
k_max = 50

method_label = (
    run_df["run_id"].dropna().unique()[0]
    if "run_id" in run_df.columns and run_df["run_id"].nunique() == 1
    else "method"
)

# On calcule la CDF sur tous les patients avec un avg_rank disponible
# (les NA comptent comme échec : ils ne contribuent pas aux <=k)
total = len(rank_eval)
if total == 0:
    raise ValueError("rank_eval est vide : aucun patient exploitable (reports absents/erreurs ?)")

cdf_data = []
for k in range(1, k_max + 1):
    cdf_k = (rank_eval["avg_rank"].notna() & (rank_eval["avg_rank"] <= k)).sum() / total
    cdf_data.append({"method": method_label, "k": k, "percentage": cdf_k * 100})

cdf_df = pd.DataFrame(cdf_data)

fig, ax = plt.subplots()

ax.plot(cdf_df["k"], cdf_df["percentage"], marker="o")
ax.set_title(f"CDF: % des cas avec rang moyen ≤ k (méthode: {method_label}, k≤{k_max})")
ax.set_xlabel("Rank k")
ax.set_ylabel("Pourcentage de cas (%)")
ax.set_ylim(0, 100)
ax.set_xticks([1, 10, 20, 30, 40, 50])
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "cdf_avg_rank.png"), dpi=300)
plt.close(fig)



## Bar chart stacked by rank

In [ ]:
def categorize_rank(rank):
    if pd.isna(rank):
        return ">50 or Not Found"
    elif rank == 1:
        return "Top 1"
    elif rank <= 5:
        return "Top 2–5"
    elif rank <= 10:
        return "Top 6–10"
    elif rank <= 20:
        return "Top 11–20"
    elif rank <= 50:
        return "Top 21–50"
    else:
        return ">50 or Not Found"

rank_order = ["Top 1", "Top 2–5", "Top 6–10", "Top 11–20", "Top 21–50", ">50 or Not Found"]

bar_df = rank_eval.copy()
bar_df["method"] = method_label
bar_df["rank_range"] = bar_df["avg_rank"].apply(categorize_rank)
bar_df["rank_range"] = pd.Categorical(bar_df["rank_range"], categories=rank_order, ordered=True)

counts = (
    bar_df.groupby(["method", "rank_range"])
    .size()
    .reset_index(name="count")
)

counts["percentage"] = counts.groupby("method")["count"].transform(lambda x: x / x.sum() * 100)

# On pivote pour avoir une colonne par tranche
pivot = counts.pivot(index="method", columns="rank_range", values="percentage")
pivot = pivot[rank_order]  # assurer l'ordre

fig, ax = plt.subplots()

bottom = np.zeros(len(pivot))
methods = pivot.index

for rr in rank_order:
    values = pivot[rr].fillna(0).values
    ax.bar(methods, values, bottom=bottom, label=rr)
    bottom += values

ax.set_title(f"Répartition des rangs moyens par tranche (méthode: {method_label})")
ax.set_xlabel("Méthode")
ax.set_ylabel("Pourcentage de cas (%)")
ax.set_ylim(0, 100)
plt.xticks(rotation=0)
ax.legend(title="Tranche de rang")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "stacked_rank_ranges.png"), dpi=300)
plt.close(fig)


## Rank by classification (ACMG)

#### 1.1 Principle and data preparation

Question: Do patients with clinically better-classified variants (ACMG/Clinvar) have higher priorization ranks?

In [ ]:
# Construction de avg_rank par patient (déjà utilisée avant)
rank_eval = (
    run_df[run_df["report_found"] & (~run_df["report_read_error"])]
    .groupby("ID_PAT_ETUDE")["rank"]
    .mean()   # ou .min() si tu veux le meilleur rang
    .reset_index(name="avg_rank")
)

# Jointure avec les métadonnées cliniques (une ligne par patient)
patient_meta = (
    run_df.drop_duplicates(subset=["ID_PAT_ETUDE"])
    [["ID_PAT_ETUDE", "acmg_classification", "clinvar_classification"]]
)

rank_by_patient = rank_eval.merge(patient_meta, on="ID_PAT_ETUDE", how="left")
rank_by_patient.head()


#### 1.2 Rank by ACMG - distribution (boxplot)

In [ ]:
fig, ax = plt.subplots()

categories = sorted(rank_by_patient["acmg_classification"].dropna().unique())
data = [
    rank_by_patient.loc[rank_by_patient["acmg_classification"] == cat, "avg_rank"].dropna()
    for cat in categories
]

ax.boxplot(data, labels=categories, showfliers=True)
ax.set_title("Distribution des rangs moyens par classification ACMG")
ax.set_xlabel("Classification ACMG")
ax.set_ylabel("Rang moyen du gène causal")
ax.invert_yaxis()  # rang 1 en haut
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "box_acmg_rank.png"), dpi=300)
plt.close(fig)


#### 1.3 Summary

In [ ]:
rank_acmg_summary = (
    rank_by_patient
    .groupby("acmg_classification")
    .agg(
        n_patients=("ID_PAT_ETUDE", "nunique"),
        mean_rank=("avg_rank", "mean"),
        median_rank=("avg_rank", "median"),
    )
    .sort_values("median_rank")
)

rank_acmg_summary

## Rank by causal gene

#### 2.1 Principle and data preparation

In [ ]:
gene_rank = (
    run_df[run_df["report_found"] & (~run_df["report_read_error"])]
    .groupby(["gene_symbol", "ID_PAT_ETUDE"])["rank"]
    .mean()
    .reset_index()
    .groupby("gene_symbol")
    .agg(
        n_patients=("ID_PAT_ETUDE", "nunique"),
        mean_rank=("rank", "mean"),
        median_rank=("rank", "median"),
    )
    .reset_index()
)

# On filtre pour éviter les gènes vus une seule fois (optionnel mais recommandé)
gene_rank_filtered = gene_rank[gene_rank["n_patients"] >= 2]

gene_rank_filtered.sort_values("mean_rank").head()

#### 2.3 Mean rank by gene barplot

In [ ]:
fig, ax = plt.subplots()

# facteur d’échelle pour la taille des points (à ajuster visuellement)
sizes = gene_rank_filtered["n_patients"] * 20  

ax.scatter(
    gene_rank_filtered["mean_rank"],
    gene_rank_filtered["gene_symbol"],
    s=sizes
)

ax.set_title("Rang moyen par gène causal (taille = nombre de patients)")
ax.set_xlabel("Rang moyen du gène causal")
ax.set_ylabel("Gène")
ax.invert_xaxis()  # comme autorange="reversed"
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "mean_rank_by_gene.png"), dpi=300)
plt.close(fig)


## Are genes with ClinVar consensus better prioritized?

##### 1.1 Data preparation

In [ ]:
run_df["clinvar_review_status"].value_counts()

In [ ]:
patient_meta = (
    run_df.drop_duplicates(subset=["ID_PAT_ETUDE"])
    [["ID_PAT_ETUDE", "gene_symbol", "clinvar_review_status", "phenotype_specificity"]]
)

rank_by_patient = rank_eval.merge(patient_meta, on="ID_PAT_ETUDE", how="left")
rank_by_patient.head()

def recode_clinvar_review(status):
    if pd.isna(status):
        return "Unknown"
    s = status.lower()
    if "single submitter" in s or "no assertion" in s:
        return "Single / Low confidence"
    elif "multiple submitters" in s or "expert panel" in s:
        return "Multiple / Expert"
    elif "conflicting" in s:
        return "Conflicting"
    else:
        return "Other"

rank_by_patient["clinvar_review_group"] = (
    rank_by_patient["clinvar_review_status"]
    .apply(recode_clinvar_review)
)

rank_by_patient["clinvar_review_group"].value_counts()


#### 1.2 Boxplot (Mean rank by Clinvar group)

In [ ]:
fig, ax = plt.subplots()

# On récupère les catégories de ClinVar dans un ordre stable
categories = sorted(rank_by_patient["clinvar_review_status"].dropna().unique())

# Données par catégorie pour le boxplot
data = [
    rank_by_patient.loc[
        rank_by_patient["clinvar_review_status"] == cat, "avg_rank"
    ].dropna()
    for cat in categories
]

ax.boxplot(data, labels=categories, showfliers=True)

ax.set_title("Rang moyen par statut de revue ClinVar")
ax.set_xlabel("ClinVar review status")
ax.set_ylabel("Rang moyen du gène causal")

# Rang 1 en haut comme avec autorange="reversed"
ax.invert_yaxis()

plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "box_clinvar_review_status.png"), dpi=300)
plt.close(fig)


In [ ]:
# Métadonnées patient (une ligne par patient)
def recode_clinvar_review(status):
    if pd.isna(status):
        return "Unknown"
    s = status.lower()
    if "single submitter" in s or "no assertion" in s:
        return "Single / Low confidence"
    elif "multiple submitters" in s or "expert panel" in s:
        return "Multiple / Expert"
    elif "conflicting" in s:
        return "Conflicting"
    else:
        return "Other"

rank_by_patient["clinvar_review_group"] = (
    rank_by_patient["clinvar_review_status"]
    .apply(recode_clinvar_review)
)

rank_by_patient["clinvar_review_group"].value_counts()


rank_by_patient = rank_eval.merge(patient_meta, on="ID_PAT_ETUDE", how="left")
rank_by_patient.head()

#### 1.2 Rank by Clinvar review status (proxy "studied gene")

In [ ]:
fig, ax = plt.subplots()

# On récupère les catégories de ClinVar dans un ordre stable
categories = sorted(rank_by_patient["clinvar_review_status"].dropna().unique())

# Données par catégorie pour le boxplot
data = [
    rank_by_patient.loc[
        rank_by_patient["clinvar_review_status"] == cat, "avg_rank"
    ].dropna()
    for cat in categories
]

ax.boxplot(data, labels=categories, showfliers=True)

ax.set_title("Rang moyen par statut de revue ClinVar")
ax.set_xlabel("ClinVar review status")
ax.set_ylabel("Rang moyen du gène causal")

# Rang 1 en haut comme avec autorange="reversed"
ax.invert_yaxis()

plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "box_clinvar_review_status.png"), dpi=300)
plt.close(fig)


# PhenoGenius specific question

#### Does prioritization performance really depend on the quality of the phenotype?

Here, we test the hypothesis by assessing the relationship between gene rank and the size of the input phenotype profile. In this step, we equate phenotype quality with the amount of information, measured by: the length of the phenotypic profile (number of HPO terms entered).

H0 (null): the rank of the causal gene is independent of the size of the phenotypic profile
H1: richer (longer) phenotypic profiles lead to better prioritization (lower rank)

#### 1.1 Phenotypic profile size construction

In [ ]:
def count_hpo(hpo_str):
    if pd.isna(hpo_str):
        return 0
    try:
        hpo_list = ast.literal_eval(hpo_str)
        return len(hpo_list)
    except Exception:
        return 0

run_df["phenotype_length"] = run_df["hpo_implicated"].apply(count_hpo)
run_df[["ID_PAT_ETUDE", "phenotype_length"]].drop_duplicates().head()

#### 1.2 Per patient rank construction

In [ ]:
rank_by_patient = (
    run_df[run_df["report_found"] & (~run_df["report_read_error"])]
    .groupby("ID_PAT_ETUDE")
    .agg(
        avg_rank=("rank", "mean"),      # ou "min" si tu préfères
        phenotype_length=("phenotype_length", "first")
    )
    .reset_index()
)

rank_by_patient.head()

#### 1.3 Correlation rank <-> phenotypic profile size

In [ ]:
fig, ax = plt.subplots()

ax.scatter(rank_by_patient["phenotype_length"], rank_by_patient["avg_rank"], alpha=0.7)
ax.set_title("Relationship between phenotype size and causal gene rank")
ax.set_xlabel("Phenotype size (number of HPO terms)")
ax.set_ylabel("Average rank of the causal gene")
ax.invert_yaxis()
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "scatter_rank_vs_phenotype_size.png"), dpi=300)
plt.close(fig)


#### 1.4 Spearman correlation

In [ ]:
from scipy.stats import spearmanr

rho, pval = spearmanr(
    rank_by_patient["phenotype_length"],
    rank_by_patient["avg_rank"],
    nan_policy="omit"
)

print(f"Spearman rho = {rho:.3f}")
print(f"P-value      = {pval:.3e}")


#### 1.5 Class size of the phenotypic profile analysis 

In [ ]:
rank_by_patient["phenotype_size_bin"] = pd.cut(
    rank_by_patient["phenotype_length"],
    bins=[0, 3, 6, 10, 20, 100],
    labels=["1–3", "4–6", "7–10", "11–20", ">20"]
)

rank_by_patient["phenotype_size_bin"].value_counts().sort_index()

fig, ax = plt.subplots()

bins = rank_by_patient["phenotype_size_bin"].cat.categories \
    if hasattr(rank_by_patient["phenotype_size_bin"], "cat") \
    else sorted(rank_by_patient["phenotype_size_bin"].dropna().unique())

data = [
    rank_by_patient.loc[rank_by_patient["phenotype_size_bin"] == b, "avg_rank"].dropna()
    for b in bins
]

ax.boxplot(data, labels=bins, showfliers=True)
ax.set_title("Causal gene rank by phenotype size")
ax.set_xlabel("Phenotype size (number of HPO terms)")
ax.set_ylabel("Average rank")
ax.invert_yaxis()
plt.tight_layout()

plt.savefig(os.path.join(OUTPUT_DIR, "box_rank_by_phenotype_size.png"), dpi=300)
plt.close(fig)


#### 1.6 Top-N indicators according to the phenotype size

Based on the phenotypic group size. Each group represents a level of phenotypic richness. And then for each patient within the group, you ask, does avg_rank <= N ?

The calcul is: Top-N% = (number of patients with rank <= N / number of total patients from group) * 100

In [ ]:
def hit_at_k(series, k):
    return (series.notna() & (series <= k)).mean() * 100

topN_by_size = (
    rank_by_patient
    .groupby("phenotype_size_bin")["avg_rank"]
    .apply(lambda s: pd.Series({
        "Top10_%": hit_at_k(s, 10),
        "Top20_%": hit_at_k(s, 20),
        "Top50_%": hit_at_k(s, 50),
    }))
)

topN_by_size


## To do

- [x] Rank by classification : Par classification, j'ai n patients avec n rangs -> rang moyen que je peux représenter ou alors même une distribution sous la forme d'un boxplot

- [x] Rank by causal gene : Par gène causal, on va avoir un rang moyen qui depend du nombre de patients, on peut afficher un barplot horizontal avec le rang moyen en valeur et le nombre de patients le partageant dessus ou sur le côté.
- [x] Les gènes très étudiés sont-ils mieux priorisés ?

- [x] Performance really depends of the quality of the phenotype? Comparer le rang par rapport à la taille du profil et à sa spécificité

- Regarder quels sont les termes HPO qui contribuent le plus
- Pourquoi la priorisation échoue ? Quand est-ce qu'elle échoue (seuil considéré) ? Est-ce un problème de phénotypes (trop faible, pas significatif), un problème de connaissance gène <-> phénotype ? (Est-ce qu'on a d'autres cas dans cette situation ?) Un problème technique ?
-  Quels termes HPO contribuent aux succès de priorisation ?
-  Quand le gène causal est mal classé, quels gènes sont bien classés ? Selon des catégories. 
-  Requête à l'échelle d'un patinet pourb les phénotypes ou barplot avec nom en dessous de la contribution des HPO dans l'outil.
-  Apporter des classifications sur les maladies ou les gènes